<a href="https://colab.research.google.com/github/falvojr/santander-dev-week-2023/blob/master/SantanderDevWeek2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Santander Dev Week 2023 (ETL com Python)

**Contexto:** Você é um cientista de dados no Santander e recebeu a tarefa de envolver seus clientes de maneira mais personalizada. Seu objetivo é usar o poder da IA Generativa para criar mensagens de marketing personalizadas que serão entregues a cada cliente.

**Condições do Problema:**

1. Você recebeu uma planilha simples, em formato CSV ('SDW2023.csv'), com uma lista de IDs de usuário do banco:
  ```
  UserID
  1
  2
  3
  4
  5
  ```
2. Seu trabalho é consumir o endpoint `GET https://sdw-2023-prd.up.railway.app/users/{id}` (API da Santander Dev Week 2023) para obter os dados de cada cliente.
3. Depois de obter os dados dos clientes, você vai usar a API do ChatGPT (OpenAI) para gerar uma mensagem de marketing personalizada para cada cliente. Essa mensagem deve enfatizar a importância dos investimentos.
4. Uma vez que a mensagem para cada cliente esteja pronta, você vai enviar essas informações de volta para a API, atualizando a lista de "news" de cada usuário usando o endpoint `PUT https://sdw-2023-prd.up.railway.app/users/{id}`.



In [72]:
import pandas as pd
x_user_id ='2154'
headers = {
    "accept": "application/json",
    "X-User-ID": x_user_id,
    "Content-Type": "application/json"
}

# Create a list of UserIDs from 1 to 16
user_ids = list(range(1, 17))

# Create a DataFrame from the list
df = pd.DataFrame({'UserID': user_ids})

# Save the DataFrame to a CSV file named 'SDW2023.csv'
df.to_csv('SDW2023.csv', index=False)

print("SDW2023.csv created successfully with UserIDs from 1 to 16.")

SDW2023.csv created successfully with UserIDs from 1 to 16.


In [ ]:
# Utilize sua própria URL se quiser ;)
# https://github.com/Luiz0tavi0/santander-dev-api-alternative
sdw2025_api_url_alternative = 'https://santander-dev-api-alternative-production.up.railway.app/'


In [68]:
#  executa o seed, para adicionar correntistas de teste
import requests


data = {
    "count": 12
}
# apaga tudo e gera 12 novos correntistas para o usuário 'x_user_id'
requests.post(url = f"{sdw2025_api_url_alternative}/seed/reset", headers=headers, json=data)

response = requests.post(url = f"{sdw2025_api_url_alternative}/users", headers=headers, json=data)

users=response.json()



## **E**xtract

Extraia a lista de IDs de usuário a partir do arquivo CSV. Para cada ID, faça uma requisição GET para obter os dados do usuário correspondente.

In [ ]:
import pandas as pd

df = pd.read_csv('SDW2023.csv')
user_ids = df['UserID'].tolist()
print(user_ids)

In [89]:
import requests
import json

def get_user(id):
  response = requests.get(f'{sdw2025_api_url_alternative}/users/{id}',headers=headers )
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

[
  {
    "name": "Bryan Moreira",
    "account": {
      "number": "00001-7",
      "agency": "1645",
      "balance": 8559.61,
      "limit": 2567.88
    },
    "features": [
      {
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/loan.svg",
        "description": "Proteja seu patrim\u00f4nio com nossos seguros"
      }
    ],
    "card": {
      "number": "**** **** **** 0001",
      "limit": 4279.81
    },
    "news": [
      {
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/insurance.svg",
        "description": "Bryan, Novas oportunidades de investimento dispon\u00edveis!"
      },
      {
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/pix.svg",
        "description": "Bryan, Proteja seu futuro com nossos planos de previd\u00eancia"
      },
      {
        "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
    

## **T**ransform

Utilize a API do Groq 'qwen/qwen3-32b' para gerar uma mensagem de marketing personalizada para cada usuário.

In [66]:
!pip install groq

In [90]:
# Substitua o texto TODO por sua API Key da OpenAI, ela será salva como uma variável de ambiente.
groq_api_key ='gsk_qualquer_coisa'

In [70]:
def sanitize_ai_text(text: str) -> str:
  """Função para remover caracteres não relevantes na resposta da I.A"""
  if not text:
    return ""

  # corta tudo antes de qualquer think
  if "<think" in text:
      text = text.split("</think>")[-1]
      text = text.split("<think")[0]

  # remove sobras
  text = text.replace("<think>", "").replace("</think>", "")

  return text.strip().strip('"').strip()


In [71]:
from pprint import pprint
import asyncio
import groq

# 2. Create the async client
client = groq.AsyncClient(api_key=groq_api_key)

# 3. Concurrency limit: max 2 simultaneous calls
sem = asyncio.Semaphore(4)


# -------------------------------------------------------------
async def generate_ai_news(user: dict) -> str:
    """
    Calls Groq to generate a short marketing message
    about investments for the given user.
    """
    async with sem:
        completion = await client.chat.completions.create(
            model="qwen/qwen3-32b",
            messages=[
                {
                    "role": "system",
                    "content": (
                        "Responda apenas o texto final em até 100 caracteres em pt-br."
                        "Você cria mensagens bancárias curtas. "
                        "Nunca revele raciocínio. Nunca use <think>. "
                        "NUNCA revele raciocínio, pensamentos ou etapas. "
                        "Responda SOMENTE com a mensagem final, no máximo 100 caracteres."
                        "As mensagens geradas devem ser em português do Brasil."
                    ),
                },
                {
                    "role": "user",
                    "content": f"Mensagem para {user['name']} sobre importância dos investimentos.",
                },
            ],
            max_tokens=25,
        )
        return completion.choices[0].message.content.strip()


# -------------------------------------------------------------
async def process_user(user: dict):
  raw_news = await generate_ai_news(user)
  news = sanitize_ai_text(raw_news)
  user["news"].append({"icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg","description": news})
  return user


# -------------------------------------------------------------
async def main():
    """
    Create tasks for all users and wait for them to finish.
    """
    tasks = [process_user(u) for u in users]
    await asyncio.gather(*tasks)


await main()


{'account': {'agency': '1665',
             'balance': 6676.89,
             'limit': 2003.07,
             'number': '00002-5'},
 'card': {'limit': 3338.45, 'number': '**** **** **** 0002'},
 'created_at': '2025-11-18T19:48:05.869276',
 'features': [{'description': 'Transfira instantaneamente com Pix',
               'icon': 'https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/invest.svg'},
              {'description': 'Invista com segurança e bons retornos',
               'icon': 'https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/insurance.svg'}],
 'name': 'Henry Gabriel Cardoso',
 'news': [{'description': 'Henry, Proteja seu futuro com nossos planos de '
                          'previdência',
           'icon': 'https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/invest.svg'},
          {'description': 'Henry, Proteja seu futuro com nossos planos de '
                          'previdência',
           'icon': 'htt

## **L**oad

Atualize a lista de "news" de cada usuário na API com a nova mensagem gerada.

In [87]:
from typing import Dict, List

async def update_user(user: Dict):
    response = requests.put(
        f"{sdw2025_api_url_alternative}/users/{user['user_id']}",
        json=user,
        headers=headers
    )
    return user, response.status_code == 200


async def update_users_batch(users: List[Dict]):
    tasks = [update_user(u) for u in users]
    results = await asyncio.gather(*tasks)

    for user, ok in results:
        if ok:
            print(f"{user['name']} atualizado com sucesso!")
        else:
            print(f"{user['name']} falhou!")

await update_users_batch(users)


Bryan Moreira atualizado com sucesso!
Henry Gabriel Cardoso atualizado com sucesso!
Anthony Gabriel Teixeira atualizado com sucesso!
Sr. Dom Rezende atualizado com sucesso!
Dr. Vinícius Montenegro atualizado com sucesso!
Ayla Rocha atualizado com sucesso!
Ana Vitória Souza atualizado com sucesso!
João Vitor Nascimento atualizado com sucesso!
Maria Julia Pereira atualizado com sucesso!
Alexandre Sousa atualizado com sucesso!
Thiago Nunes atualizado com sucesso!
Sr. Thales da Luz atualizado com sucesso!
